<a href="https://colab.research.google.com/github/bryanwills/python_codelou/blob/main/real_estate_property_info.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1 - Imports

In [1]:
!pip install requests
!pip install pandas
!pip install openpyxl
!pip install sqlalchemy
!pip install psycopg2-binary
import requests
import pandas as pd
import sqlalchemy
import openpyxl
import psycopg2
import datetime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.0 MB 4.1 MB/s 


Go to realtor.ca site, enter in a search. My example: "Ottawa, ON".
After making this search, open up DevTools, click on Network and go through the files and locate the query in the form of a curl command. Convert the curl command to a Python request and paste the output below in Step 2. I just Googled curl to python converter.

This site was chosen due to the fact that I can pull out their API from dev tools without having to sign up for an API.

Step 2 - Requests & URL


In [2]:
headers = {
    'authority': 'api2.realtor.ca',
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9,cs;q=0.8',
    'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'cookie': 'visid_incap_2269415=DRpcY4I3SjKFMfalMxSftyaK6WIAAAAAQUIPAAAAAACvrLYGe+e3wDZmQZfgEaBA; nlbi_2269415=B6YSQ7YtUCMAjnPWn2FYxQAAAACYHLiQKXa5Yo8sEYT9lH/D; incap_ses_1291_2269415=SdugIV1PHwRJVXABPo7qESaK6WIAAAAA8vWAvOTCm3r54LL1BaCVtA==; incap_ses_116_2269415=FM63c0UCPQHIXBz0Kh6cASeK6WIAAAAA9WPFdt5xRRbegjs061rpqw==; reese84=3:W3xEMitJn4FeyQaF2XIXWQ==:p7Gd6DFbHB7lgs8ZiewNBgGqjQLo3wb7ZgLM5HEqdiD5sN4Mr44u810tBQhqV2vjU2U1jZhk1XUUaxH3SE6rRKOZt2IsJBaY91rEmL/BfA9TmyLmjn+zJzFmdGtmAiChQgjCUv8dENZu7G/vy9dAv2VS2qzg0HdfHvG9N6GpzCGi9tItYSFdAC8W6Hf1v7ru/vUdI9Gswb/t3vvtWsVgd0VwG0/MuhT3PJHUsJuK826Fry8SmmvyvBkYGWDIE/+2lgErv/802OoFzhg3SoNuJAU5wWhb00pBiZGi02wn5GRKByp4s/E2VoVX6arUlcELR5JUvMfIAehQsgzhbw4HOSKf5nJaZYKSYpYhSV/lyUoqp54LPEcLOcJiPaisJj2GJPoKHsQqa2LX7yNuFJCgwYL/h/luyeyPr/f5IKQnqGA=:/enhsFfOjcbixgt8+8usYgkzfKJI11l1g99of43Onuc=; gig_bootstrap_3_mrQiIl6ov44s2X3j6NGWVZ9SDDtplqV7WgdcyEpGYnYxl7ygDWPQHqQqtpSiUfko=gigya-pr_ver4; ASP.NET_SessionId=4je5mml0zgqlvsnfaxs3egfb; visid_incap_2271082=B8sI/rNOS1O+7SuN27HKhkmK6WIAAAAAQUIPAAAAAADhnJ+CcKEEfvlCTsuuOLuJ; nlbi_2271082=shHQdkaMZgmv7s1ENo2IPAAAAAD0GfZzBej4n1ca1rAudqyl; incap_ses_116_2271082=fbUKUHhRXEGMgxz0Kh6cAUmK6WIAAAAAlXQi/sn9Rh1WHoAG/5A3fg==; nlbi_2269415_2147483392=u6DgadTQhS/ycA2tn2FYxQAAAABhO27gJcvY6w7tSWqOqotq',
    'dnt': '1',
    'origin': 'https://www.realtor.ca',
    'referer': 'https://www.realtor.ca/',
    'sec-ch-ua': '^\\^.Not/A)Brand^\\^;v=^\\^99^\\^, ^\\^Google',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '^\\^Windows^\\^',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36',
}

data = {
  'ZoomLevel': '10',
  'LatitudeMax': '45.61508',
  'LongitudeMax': '-75.15382',
  'LatitudeMin': '44.88320',
  'LongitudeMin': '-76.44608',
  'Sort': '6-D',
  'PropertyTypeGroupID': '1',
  'PropertySearchTypeId': '1',
  'TransactionTypeId': '2',
  'Currency': 'FXUSDCAD',
  'RecordsPerPage': '12',
  'ApplicationId': '1',
  'CultureId': '1',
  'Version': '7.0',
  'CurrentPage': '1'
}

response = requests.post('https://api2.realtor.ca/Listing.svc/PropertySearch_Post', headers=headers, data=data)


Step 3 - Check Status Code

In [3]:
response

<Response [200]>

Running this cell gets back a Response [200] indicating that the response from the request ran without problem.

Step 4 - Create JSON Object

In [4]:
result_json = response.json()

After running the above code cell, the data that comes back is the information we are looking for in json format.

Step 5 - Output Keys


In [5]:
result_json.keys()

dict_keys(['ErrorCode', 'Paging', 'Results', 'Pins', 'GroupingLevel'])

Storing keys from webpage (dev tools). This will allow me to extract specific data later on. Specifically, looking at the 'results' key which contains all the information I want and need.

Step 6 - Find Data


1.   Address
2.   Bedrooms
3.   Bathrooms
4.   Agent Name
5.   Area Code
6.   Phone Number
7.   Price




In [6]:
# Starting point
result_items = result_json['Results']

In [7]:
len(result_items)

12

Running the above code allows me to see the information from the results dictionary key. If I were to put len(result_json['Results']), I would get back '12' which is the same amount of search results from the website or after storing the results into a variable, I can check the length by the above code. For now, I am going to store that data in result_items variable.

In [8]:
# address of first result using the index of '0'
result_items[0]['Property']['Address']['AddressText']
# targeting the 'Property' key to get the address since it is within the property key.
# Going further to get the specific data I am looking for, I keep going down the 'keys' until I have just the street address I need.


'1556 OLD PRESCOTT ROAD|Greely, Ontario K4P1B8'

In [9]:
# Get bedroom information 
result_items[0]['Building']['Bedrooms']

'2 + 2'

In [10]:
# Get bathrooms information
result_items[0]['Building']['BathroomTotal']

'3'

In [11]:
# Get the Agent Name. Since the Agent is part of a list, need to use an index to extract the information.
result_items[0]['Individual'][0]['Name']

'Ellen Hughes'

In [12]:
# Get the area code, using index 0 since there is a fax number as well
result_items[0]['Individual'][0]['Phones'][0]['AreaCode']

'613'

In [13]:
# Get the phone number
result_items[0]['Individual'][0]['Phones'][0]['PhoneNumber']

'858-8640'

In [14]:
# extract price of property
result_items[0]['Property']['Price']

'$659,000'

Step 7 - Put it all together and loop through results appending data inside a list

In [15]:
# Create empty lists, these are the data points I want to extract from the webpage
address = []
bedrooms = []
bathrooms = []
agent_name = []
area_code = []
phone_number = []
price = []

# iterate over each item on the page
for result in result_items:

    # address
    try:
        address.append(result['Property']['Address']['AddressText'])
    except:
        address.append('')

    # bedrooms
    try:
        bedrooms.append(result['Building']['Bedrooms'])
    except:
        bedrooms.append('')

    # bathrooms
    try:
        bathrooms.append(result['Building']['BathroomTotal'])
    except:
        bathrooms.append('')

    # Agent Name
    try:
        agent_name.append(result['Individual'][0]['Name'])
    except:
        agent_name.append('')

    # area code
    try:
        area_code.append(result['Individual'][0]['Phones'][0]['AreaCode'])
    except:
        area_code.append('')

    # phone number
    try:
        phone_number.append(result['Individual'][0]['Phones'][0]['PhoneNumber'])
    except:
        phone_number.append('')

    # price
    try:
        price.append(result['Property']['Price'])
    except:
        price.append('')

In [16]:
# This will print out all 12 addresses from the search results
address



['1556 OLD PRESCOTT ROAD|Greely, Ontario K4P1B8',
 '418 COOKS MILL CRESCENT|Ottawa, Ontario K1V2M9',
 '206 PARKIN CIRCLE|Gloucester, Ontario K1T4G4',
 '336 Rue de Pointe-Gatineau|Gatineau (Gatineau), Quebec J8T2C1',
 '160 GEORGE STREET UNIT#703|Ottawa, Ontario K1N9M2',
 '215 MABEL STREET|Limoges, Ontario K0A2M0',
 '27 WARREN AVENUE|Ottawa, Ontario K1Y0R9',
 '187 PATTERSON AVENUE|Ottawa, Ontario K1S1Y4',
 '75 ANDERSON STREET|Ottawa, Ontario K1R6T6',
 '85 Rue de Rouville|#24|Gatineau (Gatineau), Quebec J8T8M7',
 '399 Rue Jeannine-Grégoire-Ross|Gatineau (Gatineau), Quebec J8P0C5',
 '132 CONSTANCE CREEK DRIVE|Ottawa, Ontario K0A1T0']

Step 8 - Pandas Dataframe

In [17]:
# Take all the data from the for loop above and put it into a Pandas Dataframe, each list should return a column
df_realtor = pd.DataFrame({'Address': address, 'Bedrooms': bedrooms, 'Bathrooms': bathrooms, 'Agent Name': agent_name, 'Area Code': area_code, 'Telephone': phone_number, 'Price': price})

In [18]:
# Display the data frame, will show all 12 addresses and related information for each listing on the page.
df_realtor

,Address,Bedrooms,Bathrooms,Agent Name,Area Code,Telephone,Price
0,"1556 OLD PRESCOTT ROAD|Greely, Ontario K4P1B8",2 + 2,3,Ellen Hughes,613,858-8640,"$659,000"
1,"418 COOKS MILL CRESCENT|Ottawa, Ontario K1V2M9",3 + 0,3,David O'Reilly,613,797-9673,"$729,000"
2,"206 PARKIN CIRCLE|Gloucester, Ontario K1T4G4",3 + 0,3,Marnie Bennett,613,233-8606,"$654,900"
3,336 Rue de Pointe-Gatineau|Gatineau (Gatineau)...,2,1,Lucie Dussault,819,328-3211,"$417,000"
4,"160 GEORGE STREET UNIT#703|Ottawa, Ontario K1N9M2",2 + 0,2,Dionne Caldwell,613,744-5525,"$579,000"
5,"215 MABEL STREET|Limoges, Ontario K0A2M0",3 + 0,3,Cedrick Gauthier,613,707-5146,"$599,900"
6,"27 WARREN AVENUE|Ottawa, Ontario K1Y0R9",3 + 0,2,Phil Labbe,613,316-3707,"$1,210,000"
7,"187 PATTERSON AVENUE|Ottawa, Ontario K1S1Y4",5 + 0,3,Julie Teskey,613,859-6599,"$1,549,000"
8,"75 ANDERSON STREET|Ottawa, Ontario K1R6T6",3 + 0,1,Frank Di Tiero,613,301-5794,"$649,900"
9,"85 Rue de Rouville|#24|Gatineau (Gatineau), Qu...",2,1,Yannick Arseneault,819,360-6967,"$399,900"


Step 9 - Multiple Pages

In [19]:
# Create empty lists, these are the data points I want to extract from the webpage
address = []
bedrooms = []
bathrooms = []
agent_name = []
area_code = []
phone_number = []
price = []

# iterate over the pages 1 to 51 using the CurrentPage key value as the variable in the for loop to get data from all the pages available. 
for i in range(1,51):

    headers = {
        'authority': 'api2.realtor.ca',
        'accept': '*/*',
        'accept-language': 'en-US,en;q=0.9,cs;q=0.8',
        'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'cookie': 'visid_incap_2269415=DRpcY4I3SjKFMfalMxSftyaK6WIAAAAAQUIPAAAAAACvrLYGe+e3wDZmQZfgEaBA; nlbi_2269415=B6YSQ7YtUCMAjnPWn2FYxQAAAACYHLiQKXa5Yo8sEYT9lH/D; incap_ses_1291_2269415=SdugIV1PHwRJVXABPo7qESaK6WIAAAAA8vWAvOTCm3r54LL1BaCVtA==; incap_ses_116_2269415=FM63c0UCPQHIXBz0Kh6cASeK6WIAAAAA9WPFdt5xRRbegjs061rpqw==; reese84=3:W3xEMitJn4FeyQaF2XIXWQ==:p7Gd6DFbHB7lgs8ZiewNBgGqjQLo3wb7ZgLM5HEqdiD5sN4Mr44u810tBQhqV2vjU2U1jZhk1XUUaxH3SE6rRKOZt2IsJBaY91rEmL/BfA9TmyLmjn+zJzFmdGtmAiChQgjCUv8dENZu7G/vy9dAv2VS2qzg0HdfHvG9N6GpzCGi9tItYSFdAC8W6Hf1v7ru/vUdI9Gswb/t3vvtWsVgd0VwG0/MuhT3PJHUsJuK826Fry8SmmvyvBkYGWDIE/+2lgErv/802OoFzhg3SoNuJAU5wWhb00pBiZGi02wn5GRKByp4s/E2VoVX6arUlcELR5JUvMfIAehQsgzhbw4HOSKf5nJaZYKSYpYhSV/lyUoqp54LPEcLOcJiPaisJj2GJPoKHsQqa2LX7yNuFJCgwYL/h/luyeyPr/f5IKQnqGA=:/enhsFfOjcbixgt8+8usYgkzfKJI11l1g99of43Onuc=; gig_bootstrap_3_mrQiIl6ov44s2X3j6NGWVZ9SDDtplqV7WgdcyEpGYnYxl7ygDWPQHqQqtpSiUfko=gigya-pr_ver4; ASP.NET_SessionId=4je5mml0zgqlvsnfaxs3egfb; visid_incap_2271082=B8sI/rNOS1O+7SuN27HKhkmK6WIAAAAAQUIPAAAAAADhnJ+CcKEEfvlCTsuuOLuJ; nlbi_2271082=shHQdkaMZgmv7s1ENo2IPAAAAAD0GfZzBej4n1ca1rAudqyl; incap_ses_116_2271082=fbUKUHhRXEGMgxz0Kh6cAUmK6WIAAAAAlXQi/sn9Rh1WHoAG/5A3fg==; nlbi_2269415_2147483392=u6DgadTQhS/ycA2tn2FYxQAAAABhO27gJcvY6w7tSWqOqotq',
        'dnt': '1',
        'origin': 'https://www.realtor.ca',
        'referer': 'https://www.realtor.ca/',
        'sec-ch-ua': '^\\^.Not/A)Brand^\\^;v=^\\^99^\\^, ^\\^Google',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '^\\^Windows^\\^',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-site',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36',
    }

    data = {
    'ZoomLevel': '10',
    'LatitudeMax': '45.61508',
    'LongitudeMax': '-75.15382',
    'LatitudeMin': '44.88320',
    'LongitudeMin': '-76.44608',
    'Sort': '6-D',
    'PropertyTypeGroupID': '1',
    'PropertySearchTypeId': '1',
    'TransactionTypeId': '2',
    'Currency': 'FXUSDCAD',
    'RecordsPerPage': '12',
    'ApplicationId': '1',
    'CultureId': '1',
    'Version': '7.0',
    'CurrentPage': str(i),
    }

    # response
    response = requests.post('https://api2.realtor.ca/Listing.svc/PropertySearch_Post', headers=headers, data=data)

    # create json object
    result_json = response.json()

    # result items
    result_items = result_json['Results']

    for result in result_items:

        # address
        try:
            address.append(result['Property']['Address']['AddressText'])
        except:
            address.append('')

        # bedrooms
        try:
            bedrooms.append(result['Building']['Bedrooms'])
        except:
            bedrooms.append('')

        # bathrooms
        try:
            bathrooms.append(result['Building']['BathroomTotal'])
        except:
            bathrooms.append('')

        # Agent Name
        try:
            agent_name.append(result['Individual'][0]['Name'])
        except:
            agent_name.append('')

        # area code
        try:
            area_code.append(result['Individual'][0]['Phones'][0]['AreaCode'])
        except:
            area_code.append('')

        # phone number
        try:
            phone_number.append(result['Individual'][0]['Phones'][0]['PhoneNumber'])
        except:
            phone_number.append('')

        # price
        try:
            price.append(result['Property']['Price'])
        except:
            price.append('')




In [20]:
# Take all the data from the for loop above and put it into a Pandas Dataframe, each list should return a column. Re-use the data frame we already created, taking to account the iteration of all the page results.
df_realtor = pd.DataFrame({'Address': address, 'Bedrooms': bedrooms, 'Bathrooms': bathrooms, 'Agent Name': agent_name, 'Area Code': area_code, 'Telephone': phone_number, 'Price': price})

In [21]:
# output the data frame to show all results. Running this cell generates 600 rows of real estate listings
df_realtor

,Address,Bedrooms,Bathrooms,Agent Name,Area Code,Telephone,Price
0,"1556 OLD PRESCOTT ROAD|Greely, Ontario K4P1B8",2 + 2,3,Ellen Hughes,613,858-8640,"$659,000"
1,"418 COOKS MILL CRESCENT|Ottawa, Ontario K1V2M9",3 + 0,3,David O'Reilly,613,797-9673,"$729,000"
2,"206 PARKIN CIRCLE|Gloucester, Ontario K1T4G4",3 + 0,3,Marnie Bennett,613,233-8606,"$654,900"
3,336 Rue de Pointe-Gatineau|Gatineau (Gatineau)...,2,1,Lucie Dussault,819,328-3211,"$417,000"
4,"160 GEORGE STREET UNIT#703|Ottawa, Ontario K1N9M2",2 + 0,2,Dionne Caldwell,613,744-5525,"$579,000"
...,...,...,...,...,...,...,...
595,"1308 ESSEX STREET|Ottawa, Ontario K1H7P2",3 + 1,4,Sarah Hunter,613,889-0171,"$1,198,000"
596,"206 Rue des Chantiers|Gatineau (Aylmer), Quebe...",4,1,Joanna Jasinska,819,351-5151,"$679,900"
597,"17 Rue du Tournoi|Gatineau (Aylmer), Quebec J9...",5,3,Marty Waite,819,665-0033,"$849,900"
598,"49 Rue de Bruxelles|#5|Gatineau (Aylmer), Queb...",2,1,Martin Chiasson,819,661-3500,"$324,900"


Step 10 - Store results in Excel

In [22]:
date = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
df_realtor.to_excel('realtor_multiple_pages.xlsx', index=False, header=True)
df_realtor.to_excel(f"realtor_pages_{date}.xlsx")


The code above stores the data frame into an Excel spreadsheet with the columns and rows specified in the code above. The filename will have realtor_pages_(current date).xlsx